In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. Charger le modèle d'embeddings sémantiques
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Charger le fichier de catégories avec exemples
ref_df = pd.read_csv("../data/categories.csv")  # adapte le chemin si besoin
ref_df.dropna(inplace=True)

# 3. Encoder chaque exemple dans un vecteur
ref_df['embedding'] = ref_df['example'].apply(lambda x: model.encode(x))

# 4. Moyenne des vecteurs par catégorie
category_embeddings = (
    ref_df.groupby('category')['embedding']
    .apply(lambda x: np.mean(x.tolist(), axis=0))
    .to_dict()
)

# 5. Encoder chaque activité du log
event_log_df['activity_vector'] = event_log_df['activity'].apply(lambda x: model.encode(str(x)))

# 6. Fonction de classification par similarité cosinus
def classify_activity(vector):
    best_cat = None
    best_score = -1
    for category, ref_vector in category_embeddings.items():
        score = cosine_similarity([vector], [ref_vector])[0][0]
        if score > best_score:
            best_score = score
            best_cat = category
    return best_cat, best_score

# 7. Appliquer à toutes les activités
results = event_log_df['activity_vector'].apply(classify_activity)
event_log_df['predicted_category'] = results.apply(lambda x: x[0])
event_log_df['similarity_score'] = results.apply(lambda x: x[1])

# 8. Nettoyage : on peut supprimer les vecteurs pour alléger
event_log_df.drop(columns=['activity_vector'], inplace=True)

# 9. Aperçu
event_log_df[['activity', 'predicted_category', 'similarity_score']].head(10)